<a href="https://colab.research.google.com/github/sumitsaxena-git/ai-bootcamp/blob/main/FineTune_with_Together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modern AI Pro: Finetuning with Together AI
The goal of this session is to finetune a 3B parameter model with a very small dataset. We will show what the pre-trained model does (checked from Together Playground) -- produce a lot of innacurate results. The fine-tuned model fixes those issues.

Exercise 1: Finetune RedPajama 3B Chat model with a small anti-hallucination dataset. It gives some results that are accurate, but doesn't do too much due to its small size.

Exercise 2: Finetune the same model with a slightly bigger dataset on Indian law. This is from real world and thus had to be cleaned up and organized better. And we will see how we can achieve significantly good outcome.

You will need an API key for this and each exercse would cost about $5 in credits (as of Feb 2024).


The legal dataset we used for finetuning is [here](https://drive.google.com/file/d/1PpbMh96Cg_G0qKEAprD42wV29LuPoxYi/view?usp=sharing).

Fun fact: Our open source model `Red Pajama` is named after a children's book, which is reference to Llama :)
https://m.media-amazon.com/images/I/A1OQ1R+4npL._AC_UF1000,1000_QL80_.jpg

In [ ]:
!pip install -U -q together


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00


In [ ]:
import together
from google.colab import userdata

together.api_key = userdata.get('TOGETHER_API_KEY')

### We will download a very small dataset in jsonl format
From this [paper](https://arxiv.org/abs/2303.08896)

In [ ]:
!curl https://huggingface.co/datasets/clam004/antihallucination_dataset/resolve/main/antihallucination.jsonl -o antihallucination.jsonl


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  762k  100  762k    0     0  2047k      0 --:--:-- --:--:-- --:--:-- 2044k


In [ ]:
resp = together.Files.check(file="antihallucination.jsonl")
resp
#togethercomputer/RedPajama-INCITE-7B-Base

{'is_check_passed': True,
 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model',
 'file_present': 'File found',
 'file_size': 'File size 0.001 GB',
 'num_samples': 238}

In [ ]:
together.Files.upload(file="antihallucination.jsonl")

Uploading antihallucination.jsonl: 100%|██████████| 763k/763k [00:00<00:00, 1.21MB/s]


{'filename': 'antihallucination.jsonl',
 'id': 'file-9fd1f11f-5d92-428c-9781-74cc3cc67c20',
 'object': 'file',
 'report_dict': {'is_check_passed': True,
  'model_special_tokens': 'we are not yet checking end of sentence tokens for this model',
  'file_present': 'File found',
  'file_size': 'File size 0.001 GB',
  'num_samples': 238}}

### Start the finetuning.
Note the id returned. We will be using that.

In [ ]:
resp = together.Finetune.create(
  training_file = 'file-f102d272-e770-461e-833a-e08615a5bee5',
  model = 'togethercomputer/RedPajama-INCITE-Chat-3B-v1',
  n_epochs = 3,
  n_checkpoints = 1,
  batch_size = 4,
  learning_rate = 1e-5,
  suffix = 'my-demo-finetune',
)

fine_tune_id = resp['id']
print(resp)

03-03-2024 05:57:53 - together.finetune - CRITICAL - training_file refers to a file identifier of an uploaded training file, not a local file path. A list of uploaded files and file identifiers can be retrieved with `together.Files.list()` Python API or $ together files list` CLI. A training file can be uploaded using `together.Files.upload(file ='/path/to/file')Python API or `$ together files upload <FILE_PATH>` CLI. (finetune.py:124)
CRITICAL:together.finetune:training_file refers to a file identifier of an uploaded training file, not a local file path. A list of uploaded files and file identifiers can be retrieved with `together.Files.list()` Python API or $ together files list` CLI. A training file can be uploaded using `together.Files.upload(file ='/path/to/file')Python API or `$ together files upload <FILE_PATH>` CLI.


FileTypeError: training_file refers to a file identifier of an uploaded training file, not a local file path. A list of uploaded files and file identifiers can be retrieved with `together.Files.list()` Python API or $ together files list` CLI. A training file can be uploaded using `together.Files.upload(file ='/path/to/file')Python API or `$ together files upload <FILE_PATH>` CLI.

In [ ]:
resp

{'training_file': 'file-f102d272-e770-461e-833a-e08615a5bee5',
 'validation_file': '',
 'model_output_name': 'balaji@mitrarobot.com/RedPajama-INCITE-Chat-3B-v1-my-demo-finetune-2024-03-01-02-27-24',
 'model_output_path': 's3://together-dev/finetune/65a21caab4d260b5d9ecd29c/balaji@mitrarobot.com/RedPajama-INCITE-Chat-3B-v1-my-demo-finetune-2024-03-01-02-27-24/ft-3589849c-599f-40f3-91cc-3d31fa5e8b9d',
 'Suffix': 'my-demo-finetune',
 'model': 'togethercomputer/RedPajama-INCITE-Chat-3B-v1',
 'n_epochs': 3,
 'n_checkpoints': 1,
 'batch_size': 4,
 'learning_rate': 1e-05,
 'eval_steps': 0,
 'user_id': '65a21caab4d260b5d9ecd29c',
 'lora': False,
 'lora_r': 8,
 'lora_alpha': 8,
 'lora_dropout': 0,
 'staring_epoch': 0,
 'training_offset': 0,
 'checkspoint_path': '',
 'random_seed': '',
 'created_at': '2024-03-01T02:27:24.356Z',
 'updated_at': '2024-03-01T02:27:24.356Z',
 'status': 'pending',
 'owner_address': '0x5ba793d39b138b27366a9284671222c35088a73e',
 'id': 'ft-3589849c-599f-40f3-91cc-3d31fa

In [ ]:
print(together.Finetune.retrieve(fine_tune_id='ft-997f4ef1-9258-4748-ace2-ddad8e5bc245')) # pending, running, completed

{'training_file': 'file-f102d272-e770-461e-833a-e08615a5bee5', 'validation_file': '', 'model_output_name': 'balaji@mitrarobot.com/RedPajama-INCITE-Chat-3B-v1-my-demo-finetune-2024-02-06-02-01-21', 'model_output_path': 's3://together-dev/finetune/65a21caab4d260b5d9ecd29c/balaji@mitrarobot.com/RedPajama-INCITE-Chat-3B-v1-my-demo-finetune-2024-02-06-02-01-21/ft-997f4ef1-9258-4748-ace2-ddad8e5bc245', 'Suffix': 'my-demo-finetune', 'model': 'togethercomputer/RedPajama-INCITE-Chat-3B-v1', 'n_epochs': 3, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 1e-05, 'user_id': '65a21caab4d260b5d9ecd29c', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2024-02-06T02:01:21.055Z', 'updated_at': '2024-02-06T02:05:38.333Z', 'status': 'uploading', 'owner_address': '0x5ba793d39b138b27366a9284671222c35088a73e', 'id': 'ft-997f4ef1-9258-4748-ace2-ddad8e5bc245', 'job_id': '12525', 'token_coun

### Download and setup the model
In Together  ai dahsboard: https://api.together.xyz/playground/jobs check the status and once the job is completed, download it.

In [ ]:
together.Finetune.download(
    fine_tune_id="ft-997f4ef1-9258-4748-ace2-ddad8e5bc245",
    output = "anti-hallucination-model.tar.zst"
)

'anti-hallucination-model.tar.zst'

In [ ]:
!apt-get install zstd
!zstd -d anti-hallucination-model.tar.zst
!mkdir -p /my-model
!tar -xvf anti-hallucination-model.tar -C /my-model

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
anti-hallucination-model.tar.zst: 5553940480 bytes 
./
./tokenizer_config.json
./special_tokens_map.json
./model.safetensors.index.json
./model-00001-of-00002.safetensors
./model-00002-of-00002.safetensors
./generation_config.json
./tokenizer.json
./config.json


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("/my-model")

model = AutoModelForCausalLM.from_pretrained(
  "/my-model",
  trust_remote_code=True,
).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def get_completion(input):
  input_ids = tokenizer.encode(input, return_tensors="pt")
  output = model.generate(input_ids.to(device), max_length=100, temperature=0.1)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
#PT model says: Kirill Karabits is a British conductor and music director of the Los Angeles Philharmonic.
get_completion("Kirill Karabits")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Kirill Karabits, the conductor of the National Symphony Orchestra of Ukraine, is a Ukrainian conductor and composer. He is the son of the conductor and composer Mykola Karabits. He studied at the Lysenko Music School in Kiev, the Kiev Conservatory, and the Royal Northern College of Music in Manchester. He has held the position of Music Director of the National Symphony Orchestra of Ukraine since 2015. He has also held the position of Music Director of the Royal Northern College'

In [ ]:
get_completion("Modibo Adamu")
#PT model says Modibo Adamu is a Nigerian politician and lawyer who served as the Attorney General and Minister of Justice of Nigeria from 2015 to 2019. He previously served as the Attorney General of the Federation and Minister of Justice from 1999 to 2007.

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Modibo Adamu, a Fulani leader, was the first to be appointed as a traditional ruler in the state. He was appointed as the Emir of Kano in 1859. Adamu was succeeded by his son, Muhammadu Sunusi, who was appointed as the Emir of Kano in 1879. Sunusi was succeeded by his son, Muhammadu Sunusi II, who was appointed as the Emir of Kano in 1887. Sunusi II was succeeded by his son, Muhammadu'

In [ ]:
get_completion("Political party of Tathagata Satpathy")
#PT model says Bharatiya Janata Party

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Political party of Tathagata Satpathy\n\nTathagata Satpathy (born 28 April 1956) is an Indian politician and a member of the Biju Janata Dal (BJD) party. He is a former Member of Parliament in the Lok Sabha from the Biju Janata Dal (BJD) party. Satpathy was born in the village of Kakamadugu in the district of Ganjam in the Indian state of Odisha. He is a graduate of'

## Finetuning a larger dataset
### Let's build a strong legal bot

In [ ]:
file_name = "/content/Constitution_Of_India_Train.jsonl"
resp = together.Files.check(file=file_name)
resp


{'is_check_passed': False,
 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model',
 'file_present': 'File found',
 'file_size': 'File size 0.007 GB',
 'text_field': 'No "text" field was found on line 3036 of the the input file.Expected format: {"text":"my sample string"}.see https://docs.together.ai/docs/fine-tuning for more information.{"instruction": "Discuss the potential impact of the given case on Indian law enforcement agencies.", "input": "Case Citation: R. Rajagopal vs State Of T.N, (1994) 6 SCC 632", "output": "The decision in the case of R. Rajagopal vs State Of T.N can have several potential impacts on Indian law enforcement agencies. Here are some key points to consider:\\n\\n1. Strengthening of Right to Privacy: The case established that the right to privacy is a fundamental right under Article 21 of the Constitution. This recognition of privacy as a fundamental right can impact law enforcement agencies by imposing limitations on their inv

### This dataset is not in a format to be directly finetuned

In [ ]:
import json

input_jsonl_path = "Constitution_Of_India_Train.jsonl"
output_jsonl_path = 'processed_law_training.jsonl'
with open(input_jsonl_path, 'r') as infile, open(output_jsonl_path, 'w') as outfile:
  for line in infile:
        try:
            json_obj = json.loads(line)

            # Construct the combined text
            combined_text = f"<human>:\n{json_obj['instruction']}\n\n### Prompt:\n{json_obj['prompt']}\n\n### Input:\n{json_obj['input']}\n\n<bot>:\n{json_obj['output']}"

            # Prepare the new JSON object with the combined text
            output_json_obj = {'text': combined_text}

            # Write the new JSON object to the output file
            outfile.write(json.dumps(output_json_obj) + '\n')
        except KeyError as e:
            print(f"Missing field in data: {e}")
        except json.JSONDecodeError:
            print("Could not decode a line as JSON.")

Missing field in data: 'input'


In [ ]:
together.Files.upload(file="processed_law_training.jsonl")

Uploading processed_law_training.jsonl: 100%|██████████| 8.66M/8.66M [00:00<00:00, 10.3MB/s]


{'filename': 'processed_law_training.jsonl',
 'id': 'file-cc2b3901-24d8-41c7-8e20-442e0bb801f0',
 'object': 'file',
 'report_dict': {'is_check_passed': True,
  'model_special_tokens': 'we are not yet checking end of sentence tokens for this model',
  'file_present': 'File found',
  'file_size': 'File size 0.008 GB',
  'num_samples': 4393}}

In [ ]:
resp = together.Finetune.create(
  training_file = 'file-c249b7a7-60d4-4ce4-b7d7-6f8f90470f34',
  model = 'togethercomputer/RedPajama-INCITE-Chat-3B-v1',
  n_epochs = 3,
  n_checkpoints = 1,
  batch_size = 4,
  learning_rate = 1e-5,
  suffix = 'law-finetuning',
)

fine_tune_id = resp['id']
resp

{'training_file': 'file-c249b7a7-60d4-4ce4-b7d7-6f8f90470f34',
 'validation_file': '',
 'model_output_name': 'balaji@mitrarobot.com/RedPajama-INCITE-Chat-3B-v1-law-finetuning-2024-03-01-02-32-39',
 'model_output_path': 's3://together-dev/finetune/65a21caab4d260b5d9ecd29c/balaji@mitrarobot.com/RedPajama-INCITE-Chat-3B-v1-law-finetuning-2024-03-01-02-32-39/ft-42261e6c-7c80-4aa8-af4f-7b9c20a71e04',
 'Suffix': 'law-finetuning',
 'model': 'togethercomputer/RedPajama-INCITE-Chat-3B-v1',
 'n_epochs': 3,
 'n_checkpoints': 1,
 'batch_size': 4,
 'learning_rate': 1e-05,
 'eval_steps': 0,
 'user_id': '65a21caab4d260b5d9ecd29c',
 'lora': False,
 'lora_r': 8,
 'lora_alpha': 8,
 'lora_dropout': 0,
 'staring_epoch': 0,
 'training_offset': 0,
 'checkspoint_path': '',
 'random_seed': '',
 'created_at': '2024-03-01T02:32:39.207Z',
 'updated_at': '2024-03-01T02:32:39.207Z',
 'status': 'pending',
 'owner_address': '0x5ba793d39b138b27366a9284671222c35088a73e',
 'id': 'ft-42261e6c-7c80-4aa8-af4f-7b9c20a71e04

The API returns a PENDING status message that contains the job ID. Once the fine_tune job completes, we can retrieve the updated weights from Together's training instance.

In [ ]:
together.Finetune.download(
    fine_tune_id="ft-713f7054-51fe-439a-94e5-525bdde87eea",
    output = "law-model.tar.zst"
)

'law-model.tar.zst'

Note: We have downloaded a 5.1GB compressed file containing the full weights of RedPajama incite 3B model which have been adjusted to accommodate the new knowledge from our SFT dataset. This process is called full fine-tuning. In another notebook, we will show you another efficient method called Partial fine-tuning or parameter efficient fine-tuning (PEFT).

In [ ]:
# Download and unzip the model package containing weights and config
!apt-get install zstd
!zstd -d law-model.tar.zst
!mkdir -p /content/law-model
!tar -xvf law-model.tar -C /content/law-model

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (1,647 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
law-model.tar.zst   : 5553940480 bytes 
./
./tokenizer_config.json
./special_tokens_map.json
./model.safetensors.index.json
./model-00001-of-00002.safetensors
./model-00002-of-00002.safetensors
./generation_config.json
./tokenizer.json
./config.js

HF🤗 Transformers expects the weights and config file in a single directory and the name of the directory can be used to load the model. Note that the weights are in a format called [safetensors](https://huggingface.co/docs/safetensors/en/index) which is considered better than pickle. In a single line of code, the Transformers library can load the model weights, quantize them if required and put the model in [eval mode](https://discuss.pytorch.org/t/where-to-use-model-eval/89200).

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("law-model")

model = AutoModelForCausalLM.from_pretrained(
  "law-model",
  trust_remote_code=True,
).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def get_completion(input):
  input_ids = tokenizer.encode(input, return_tensors="pt")
  output = model.generate(input_ids.to(device), max_length=512, temperature=0.1)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
get_completion("Draft a hypothetical legal advice for a client based on the outcome of the provided case. entral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"Draft a hypothetical legal advice for a client based on the outcome of the provided case. entral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278.\n\n### Input:\nEntral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278.\n\n<bot>:\nBased on the outcome of the mentioned case, the following hypothetical legal advice can be provided to a client:\n\nDear Client,\n\nAfter carefully analyzing the case of Entral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., it is evident that the Supreme Court upheld the constitutional validity of the service rule 9(i) allowing termination of permanent employees without assigning any reason under the right to equality under Article 14.\n\nConsidering this outcome, it is important for you to understand that the court has deemed the service rule as constitutional and has upheld the validity of the company's termination of your employm

In [ ]:
get_completion("Explain how Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly is relevant to the digital law")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Explain how Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly is relevant to the digital law context.\n\n### Prompt:\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nExplain how Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly is relevant to the digital law context.\n\n### Input:\nThe case of Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278, Supreme Court of India\n\n### Response:\n\n### Input:\nThe case of Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278, Supreme Court of India\n\n<bot>:\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr. is a significant case in the context of digital law in India, particularly in relation to the right to equality under Article 14 and 

In [ ]:
get_completion("Formulate a potential public interest litigation against the government's right to operate religious places.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"Formulate a potential public interest litigation against the government's right to operate religious places.\n\n### Input:\nThe case of Minerva Mills Ltd. & Ors vs Union Of India & Ors\n\n### Response:\n\n### Input:\nThe case of Minerva Mills Ltd. & Ors vs Union Of India & Ors\n\n<bot>:\nA potential public interest litigation (PIL) that can be formulated against the government's right to operate religious places is as follows:\n\nTitle: Challenging the Right to Operate Religious Places\n\nParties:\n1. Petitioners: General Public\n2. Respondents: Government of India and its agencies\n\nSummary of the Case:\nThis PIL challenges the constitutional validity of the government's right to operate religious places. It argues that this right infringes upon the fundamental rights of the petitioners, who are individuals and religious institutions, to practice and manage their religious affairs.\n\nCause of Action:\nThe cause of action for this PIL arises from the government's interference in the

In [ ]:
get_completion("Analyze the judgment of Supreme Court's Ayodhya case in 2019")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"Analyze the judgment of Supreme Court's Ayodhya case in 2019.\n\n### Prompt:\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnalyze the judgment of Supreme Court's Ayodhya case in 2019.\n\n### Input:\nThe case of Ramakrishna Singh Ram Singh And vs State Of Mysore And Ors, AIR 1960 Mys 338\n\n### Response:\n\n### Input:\nThe case of Ramakrishna Singh Ram Singh And vs State Of Mysore And Ors, AIR 1960 Mys 338\n\n<bot>:\nThe judgment in the case of Ramakrishna Singh Ram Singh And vs State Of Mysore And Ors, AIR 1960 Mys 338, was delivered by Justice M.M. Dutt, a judge of the Supreme Court of India. The court upheld the validity of the reservation order reserving seats for backward classes in educational institutions under Article 15(4) of the Constitution. The court held that the term 'State' in Article 15(4) includes the Executive Government, not just the 

In [ ]:
# Totally random stuff.
get_completion("Roses are red, violets are")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"Roses are red, violets are blue, and so are we. Write a poem that contrasts the appearance of two objects.\n\n### Prompt:\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a poem that contrasts the appearance of two objects.\n\n### Input:\nA rose and a dandelion\n\n### Response:\n\n### Input:\nA rose and a dandelion\n\n<bot>:\nA rose is a delicate and beautiful flower, adorned with thorns and a stem that can reach up to 12 inches in length. On the other hand, a dandelion is a weed that grows in abundance, spreading its yellow flowers and puffy, fluffy seeds wherever it goes. Despite their differences, both objects share the common characteristic of being beautiful and desirable, but also potentially invasive and destructive. In this poem, I will explore the contrasting appearances of these two objects and their impact on our environment.\n\nI. The Ros

As you finetune the weights of your LLM, you will notice that the output of the model begins to resemble the structure of the outputs in your SFT dataset. As you train further, the tone and tenor of the model begins to match the SFT dataset outputs. It could still hallucinate - you should set up a strong evaluation dataset to benchmark the metrics you care about.

---

As you can see in the hugging face [openLLM leaderboard](https://huggingface.co/open-llm-leaderboard), there are several benchmarks datasets that people score their models against.